# **Video Segmentation with SAM 3**

Author: Dr.Yusuke Kimata (ORCID: https://orcid.org/0000-0002-1366-0636)<br>
ref: Notebook by Meta Research (https://github.com/facebookresearch/sam3/tree/main/examples)<br>
Notebook by Roboflow (https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/how-to-segment-videos-with-segment-anything-3.ipynb)

## Prerequisite
Ensure you have a directory named `input` containing your video frames as JPEG files (e.g., `000.jpg`, `001.jpg`, ...).<br>
Make sure to change the runtime type to **"GPU"**

## 1. Environment Setup

### Configure your API keys

To pull Segment Anything 3 weights, you need a HuggingFace Access Token with approved access to the SAM 3 checkpoints.

- Request access to the SAM 3 checkpoints on the official Hugging Face [repo](https://huggingface.co/facebook/sam3).
- Open your HuggingFace Settings page. Click Access Tokens then New Token to generate a new token.
- In Colab, go to the left pane and click on Secrets (🔑). Store your HuggingFace Access Token under the name `HF_TOKEN`.



In [ ]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Check GPU availability


In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import torch
import torchvision

print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

### Install SAM 3 and dependencies

If you run this cell, you will be prompted to restart the session. Approve the restart, then proceed to the next cell.

In [ ]:
import os

!pip install -q supervision roboflow

# Clone and install SAM 3
!git clone https://github.com/facebookresearch/sam3.git # Comment out this line on subsequent runs
%cd sam3
!pip install -q -e ".[notebooks]"
%cd /content


When you run the next cell, a “Session crashed” message may appear. You can ignore it and proceed to the next cell.

In [ ]:
os._exit(0)

In [ ]:
!pip install -q supervision jupyter_bbox_widget

In [ ]:
!pip uninstall -y cc_torch; TORCH_CUDA_ARCH_LIST="8.0 9.0"; pip install git+https://github.com/ronghanghu/cc_torch
!pip uninstall -y torch_generic_nms; TORCH_CUDA_ARCH_LIST="8.0 9.0"; pip install git+https://github.com/ronghanghu/torch_generic_nms

In [ ]:
import cv2
import torch
import os

import numpy as np
import supervision as sv

from pathlib import Path
from PIL import Image
from typing import Optional
from IPython.display import Video
import matplotlib.pyplot as plt

from jupyter_bbox_widget import BBoxWidget

from sam3.model_builder import build_sam3_video_predictor
from sam3.visualization_utils import (
    prepare_masks_for_visualization,
    visualize_formatted_frame_output,
)

import ipywidgets as widgets
from IPython.display import display

HOME = Path.cwd()
print("HOME:", HOME)

In [ ]:
input_name = "input"
output_name = "output"

INPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/SAM3/" + input_name + "/"
OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/SAM3/" + output_name + "_mask/"

if not os.path.exists(INPUT_DIR):
    os.makedirs(INPUT_DIR)
    print(f"Created {INPUT_DIR}. Please upload your frames here.")
    # Create a dummy video frame for testing if empty
    img = np.zeros((512, 512, 3), dtype=np.uint8)
    cv2.circle(img, (256, 256), 50, (0, 255, 0), -1) # Green circle
    cv2.imwrite(os.path.join(INPUT_DIR, "000.jpg"), img)
    cv2.circle(img, (260, 260), 50, (0, 255, 0), -1) # Moved circle
    cv2.imwrite(os.path.join(INPUT_DIR, "001.jpg"), img)
else:
    print(f"Directory {INPUT_DIR} exists.")

os.makedirs(OUTPUT_DIR, exist_ok=True)

## 2. Configuration & Model Loading

In [ ]:
# Setup Device
DEVICE = [torch.cuda.current_device()]

BPE_PATH = "/content/sam3/sam3/assets/bpe_simple_vocab_16e6.txt.gz"

# Load Model
# Pass checkpoint_path, bpe_path, and config_file explicitly as keyword arguments
predictor = build_sam3_video_predictor(
    bpe_path=BPE_PATH,
    gpus_to_use=DEVICE
)

## 3. Data Loading

In [ ]:
frame_paths = sorted([os.path.join(INPUT_DIR, f) for f in os.listdir(INPUT_DIR) if f.lower().endswith(('.jpg', '.jpeg'))])
print(f"Found {len(frame_paths)} frames.")

# Preview first frame
if frame_paths:
    sv.plot_image(cv2.imread(frame_paths[0]), (5,5))

## 4. Inference loop

In [ ]:
response = predictor.handle_request(
    request=dict(
        type="start_session",
        resource_path=Path(INPUT_DIR).as_posix(),
    )
)
session_id = response["session_id"]

In [ ]:
_ = predictor.handle_request(
    request=dict(
        type="reset_session",
        session_id=session_id,
    )
)

### Adding a text prompt

In [ ]:
def load_frame(directory: str, index: int):
    """
    Loads a frame with a specific index from a directory where frames are named
    using the pattern '%03d.jpg' (e.g., 000.jpg, 001.jpg, 002.jpg).

    Args:
        directory (str): Path to the directory containing image frames.
        index (int): Frame index (0-based).

    Returns:
        numpy.ndarray: Loaded frame in BGR format.

    Raises:
        FileNotFoundError: If the frame does not exist or cannot be read.
    """
    directory_path = Path(directory)
    frame_path = directory_path / f"{index:03d}.jpg"

    if not frame_path.exists():
        raise FileNotFoundError(f"Frame not found: {frame_path}")

    frame = cv2.imread(str(frame_path))
    if frame is None:
        raise FileNotFoundError(f"Failed to load frame: {frame_path}")

    return frame


Specify the object you want to detect using a TEXT prompt (e.g., "cell"). SAM 3 will automatically identify multiple object instances and assign each a unique object ID.

In [ ]:
frame_idx = 0
frame = load_frame(INPUT_DIR, frame_idx)

TEXT = "cell"

### Segmentation in the reference frame

In [ ]:
response = predictor.handle_request(
    request=dict(
        type="add_prompt",
        session_id=session_id,
        frame_index=frame_idx,
        text=TEXT,
    )
)
result = response["outputs"]

### Visualize results

In [ ]:
def from_sam(result: dict) -> sv.Detections:
    return sv.Detections(
        xyxy=sv.mask_to_xyxy(result["out_binary_masks"]),
        mask=result["out_binary_masks"],
        confidence=result["out_probs"],
        tracker_id=result["out_obj_ids"],
    )

In [ ]:
COLOR = sv.ColorPalette.from_hex([
    "#ffff00", "#ff9b00", "#ff8080", "#ff66b2", "#ff66ff", "#b266ff",
    "#9999ff", "#3399ff", "#66ffff", "#33ff99", "#66ff66", "#99ff00"
])

def annotate(image: np.ndarray, detections: sv.Detections, text = None) -> np.ndarray:
    h, w, _ = image.shape
    text_scale = sv.calculate_optimal_text_scale(resolution_wh=(w, h)) * 1.5

    mask_annotator = sv.MaskAnnotator(
        color=COLOR,
        color_lookup=sv.ColorLookup.TRACK,
        opacity=0.3
    )

    annotated_image = image.copy()
    annotated_image = mask_annotator.annotate(annotated_image, detections)

    if text:
        label_annotator = sv.LabelAnnotator(
            color=COLOR,
            color_lookup=sv.ColorLookup.TRACK,
            text_scale=text_scale,
            text_color=sv.Color.BLACK,
            text_position=sv.Position.TOP_CENTER,
            text_offset=(0, -5)
        )
        labels = [
            f"#{tracker_id} {text} {confidence:.2f}"
            for tracker_id, confidence in zip(detections.tracker_id, detections.confidence)
        ]
        annotated_image = label_annotator.annotate(annotated_image, detections, labels)

    return annotated_image

In [ ]:
detections = from_sam(result)

annotated_frame = annotate(frame, detections, TEXT)

print("Label structure: [OBJECT ID] [TEXT] [CONFIDENCE]")
sv.plot_image(annotated_frame, size=(5, 5))

In [ ]:
# plt.close("all")
# visualize_formatted_frame_output(
#     frame_idx,
#     frame_paths,
#     outputs_list=[prepare_masks_for_visualization({frame_idx: result})],
#     titles=["SAM 3 Dense Tracking outputs"],
#     figsize=(5, 5),
# )

### Propagate in video

In [ ]:
def propagate_in_video(predictor, session_id):
    frame_outputs = {}
    for response in predictor.handle_stream_request(
        request=dict(
            type="propagate_in_video",
            session_id=session_id,
        )
    ):
        frame_outputs[response["frame_index"]] = response["outputs"]

    return frame_outputs

In [ ]:
frame_outputs = propagate_in_video(predictor=predictor, session_id=session_id)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def view_result(frame_idx):
    if frame_idx not in frame_outputs:
        return

    # Load the original frame
    try:
        frame = load_frame(INPUT_DIR, frame_idx)
    except FileNotFoundError:
        print(f"Could not load frame {frame_idx}")
        return

    # Get SAM output and convert to detections
    output = frame_outputs[frame_idx]
    detections = from_sam(output)

    # Annotate using the previously defined function
    # Use the global TEXT variable if it exists
    label_text = TEXT if 'TEXT' in globals() else None
    annotated_frame = annotate(frame, detections, label_text)

    # Print frame specific results for verification
    if len(detections.confidence) > 0:
        print(f"Frame {frame_idx} Results:")
        for tracker_id, confidence in zip(detections.tracker_id, detections.confidence):
            print(f" - ID {tracker_id}: Confidence {confidence:.4f}")
    else:
        print(f"Frame {frame_idx}: No detections.")

    # Display
    sv.plot_image(annotated_frame, size=(6, 6))

# Create the interactive widget
if 'frame_outputs' in globals() and frame_outputs:
    indices = sorted(frame_outputs.keys())
    min_val, max_val = indices[0], indices[-1]

    slider = widgets.IntSlider(
        value=min_val,
        min=min_val,
        max=max_val,
        step=1,
        description='Frame:'
    )

    print("Interact with the slider to view processed frames:")
    widgets.interact(view_result, frame_idx=slider)
else:
    print("No frame outputs found. Please run the propagation step first.")

## 5. Save results

### Save as binary mask

In [ ]:
# Set to specific ID (e.g., 0, 1) to save only that object's mask.
# Set to None to merge all detected objects into one mask.
TARGET_OBJECT_ID = None

print(f"Saving binary masks to {OUTPUT_DIR}...")
if TARGET_OBJECT_ID is not None:
    print(f"Targeting Object ID: {TARGET_OBJECT_ID}")
else:
    print("Targeting: All Objects (Merged)")

# Iterate through processed frames
for frame_idx, output in sorted(frame_outputs.items()):
    # Get the binary masks (Shape: [N, H, W]) and object IDs
    masks = output["out_binary_masks"]
    obj_ids = output["out_obj_ids"]

    final_mask = None

    if TARGET_OBJECT_ID is not None:
        # Find index of the target object
        indices = np.where(obj_ids == TARGET_OBJECT_ID)[0]
        if len(indices) > 0:
            final_mask = masks[indices[0]]
    else:
        # Combine masks for all objects into a single binary mask
        if masks.shape[0] > 0:
            final_mask = np.any(masks, axis=0)

    # Skip if no mask found for this frame
    if final_mask is None:
        continue

    # Convert boolean mask to 8-bit integer (0 -> 0, True -> 255)
    mask_image = final_mask.astype(np.uint8) * 255

    # Construct filename (e.g., 000.jpg)
    file_path = os.path.join(OUTPUT_DIR, f"{frame_idx:03d}.jpg")

    # Save image
    cv2.imwrite(file_path, mask_image)

print(f"Saved mask images.")

### Save as merged image

In [ ]:
# Set to specific ID (e.g., 0, 1) to save only that object's mask.
# Set to None to merge all detected objects into one image.
# Note: This uses the value set in the previous cell unless redefined here.
# TARGET_OBJECT_ID = 0

# Define the output directory for merged images
MERGED_DIR = "/content/drive/MyDrive/Colab Notebooks/SAM3/" + output_name + "_merged/"

# Create the directory if it doesn't exist
os.makedirs(MERGED_DIR, exist_ok=True)

print(f"Saving merged frames to {MERGED_DIR}...")

# Ensure TARGET_OBJECT_ID exists
if 'TARGET_OBJECT_ID' not in globals():
    TARGET_OBJECT_ID = None

if TARGET_OBJECT_ID is not None:
    print(f"Targeting Object ID: {TARGET_OBJECT_ID}")
else:
    print("Targeting: All Objects")

# Initialize MaskAnnotator with fixed opacity
mask_annotator = sv.MaskAnnotator(
    color=COLOR,
    color_lookup=sv.ColorLookup.TRACK,
    opacity=0.3
)

# Iterate through all processed frames
for frame_idx, output in sorted(frame_outputs.items()):
    # Load the original frame using the helper function
    try:
        frame = load_frame(INPUT_DIR, frame_idx)
    except FileNotFoundError:
        print(f"Warning: Could not load frame {frame_idx}, skipping.")
        continue

    # Convert the SAM output to supervision Detections
    detections = from_sam(output)

    # Filter detections if a specific object ID is targeted
    if TARGET_OBJECT_ID is not None:
        detections = detections[detections.tracker_id == TARGET_OBJECT_ID]

    # Annotate the frame with the masks using our custom annotator
    merged_frame = mask_annotator.annotate(frame.copy(), detections)

    # Construct the output filename (e.g., 000.jpg)
    save_path = os.path.join(MERGED_DIR, f"{frame_idx:03d}.jpg")

    # Save the merged image
    cv2.imwrite(save_path, merged_frame)

print(f"Successfully saved {len(frame_outputs)} merged images.")

## 6. Close session

Each session is tied to a single video. We can close the session after inference to free up its resources.

(Then, you may start a new session on another video.)

In [ ]:
_ = predictor.handle_request(
    request=dict(
        type="close_session",
        session_id=session_id,
    )
)

predictor.shutdown()